In [2]:
import math as m
import numpy as np
from numpy.fft import fft,ifft

# convoulução circular usando FFT
def cconv(x,h):
  hpad = np.zeros(len(x))
  hpad[:len(h)] = h
  return np.real(ifft(fft(x)*fft(hpad)))

In [6]:
# DWT ortogonal de Haar com M estágios
# OBs: essa mesma implementação serve para outros bancos de filtros, sendo que, se
# forem causais, apenas as duas linhas que definem la e ha precisam ser alteradas.
# Se os filtros não forem causais, será necessário fazer um shift circiular das saídas
# das convoluções usando a função np.roll(..., -L), onde L é o número de coeficientes
# do filtro com índices negativos
def dwt_haar(x, M=1):
  # filtros de análise de Haar (definidos para n=0,1)
  c = m.sqrt(2)/2
  la = np.array([c,c]) # filtro passa-baixas
  ha = np.array([c,-c]) # filtro passa-altas
  # inicializa vetor X de saída com uma cópia de x
  N = len(x)
  X = np.ndarray(np.shape(x))
  X[:] = x
  # em cada estágio j, codificaremos o vetor
  # y = X_{ll...l} = coefs de aproximação do estágio anterior
  for j in range(M):
    y = X.copy()[:N] # seleciona o trecho a codificar
    X[:N//2] = cconv(y,la)[::2] # Xl = D(x*l)
    X[N//2:N] = cconv(y,ha)[::2] # Xh = D(x*h)
    N = N//2 # próximo estágio processará metade do vetor
  return X

In [ ]:

# IDWT ortogonal de Haar com M estágios
# Obs: essa mesma implementação serve para outros bancos de filtros. Para o banco
# de filtros de Haar, cada filtro de síntese possui L=1 coeficientes com índices
# negativos, e por isso as saídas das convoluções são corrigidas por um shift
# circular usando a função np.roll(..., L).
def idwt_haar(X, M=1):
  # filtros de análise de Haar (definidos para n=-1,0)
  c = m.sqrt(2)/2
  ls = np.array([c,c]) # filtro passa-baixas
  hs = np.array([-c,c]) # filtro passa-altas
  # inicializa vetor x de saída
  x = X.copy()
  # começa a reconstrução do último nível
  n = len(X)//2**M
  for j in range(M,0,-1):
    # coeficientes de aproximação e detalhes do nível j
    cA, cD = x[:n], x[n:2*n]
    # superamostragem
    UXl = np.zeros(2*n); UXl[::2] = cA
    UXh = np.zeros(2*n); UXh[::2] = cD
    # filtragem (a função np.roll ajusta o shift dos filtros de síntese)
    vl, vh = np.roll(cconv(UXl, ls), -1), np.roll(cconv(UXh, hs), -1)
    x[:2*n] = vl+vh # combina canais
    n = n*2 # próximo nível terá o dobro dos coeficientes
  return X